# EDA: Обзор данных для обнаружения мошенничества

Этот ноутбук содержит:
- **Задачи 11-15**: Data Setup (загрузка, проверка структуры, дубликаты, пропуски)
- **Задачи 16-18**: EDA (анализ распределений amount, времени, баланса классов)
- **Задачи 19-22**: EDA продолжение (анализ по пользователям, устройствам/IP, визуализации, корреляция)


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# --- КОНСТАНТЫ ---
TRANS_FILE = '../data/транзакции в Мобильном интернет Банкинге.csv'
BEHAV_FILE = '../data/поведенческие паттерны клиентов.csv'
ENCODING = 'cp1251'
DELIMITER = ';'
DATE_FORMAT_SHORT = '%Y-%m-%d'

print("=" * 60)
print("EDA: ОБЗОР ДАННЫХ ДЛЯ ОБНАРУЖЕНИЯ МОШЕННИЧЕСТВА")
print("=" * 60)


## ЗАДАЧА 11: Найти и загрузить датасет


In [ ]:
# Загрузка данных

try:
    df_trans = pd.read_csv(TRANS_FILE, encoding=ENCODING, sep=DELIMITER, header=1)
    df_behav = pd.read_csv(BEHAV_FILE, encoding=ENCODING, sep=DELIMITER)
    print(f" Транзакции загружены: {df_trans.shape}")
    print(f" Поведенческие данные загружены: {df_behav.shape}")
except FileNotFoundError as e:
    print(f" ОШИБКА: Файл не найден - {e}")
    raise
except Exception as e:
    print(f" ОШИБКА при загрузке: {e}")
    raise


## ЗАДАЧА 12: Проверить структуру данных


In [ ]:

print("=" * 60)
print("ТАБЛИЦА ТРАНЗАКЦИЙ")
print("=" * 60)
print(f"Размер: {df_trans.shape}")
print(f"\nКолонки ({len(df_trans.columns)}):")
for i, col in enumerate(df_trans.columns, 1):
    print(f"  {i}. {col}")
print(f"\nТипы данных:")
print(df_trans.dtypes)
print(f"\nПервые 3 строки:")
print(df_trans.head(3))

print("\n" + "=" * 60)
print("ТАБЛИЦА ПОВЕДЕНЧЕСКИХ ПАТТЕРНОВ")
print("=" * 60)
print(f"Размер: {df_behav.shape}")
print(f"\nКолонки ({len(df_behav.columns)}):")
for i, col in enumerate(df_behav.columns, 1):
    print(f"  {i}. {col}")
print(f"\nТипы данных:")
print(df_behav.dtypes)
print(f"\nПервые 3 строки:")
print(df_behav.head(3))


## ЗАДАЧА 13: Удалить дубликаты


In [ ]:

# Проверка дубликатов в транзакциях
duplicates_trans = df_trans.duplicated().sum()
print(f"Дубликаты в транзакциях: {duplicates_trans}")
if duplicates_trans > 0:
    print(f"  Процент дубликатов: {duplicates_trans / len(df_trans) * 100:.2f}%")
    df_trans = df_trans.drop_duplicates()
    print(f" Дубликаты удалены. Новый размер: {df_trans.shape}")
else:
    print(" Дубликатов не найдено")

# Проверка дубликатов в поведенческих данных
duplicates_behav = df_behav.duplicated().sum()
print(f"\nДубликаты в поведенческих данных: {duplicates_behav}")
if duplicates_behav > 0:
    print(f"  Процент дубликатов: {duplicates_behav / len(df_behav) * 100:.2f}%")
    df_behav = df_behav.drop_duplicates()
    print(f" Дубликаты удалены. Новый размер: {df_behav.shape}")
else:
    print(" Дубликатов не найдено")


## ЗАДАЧА 14: Проверить пропущенные значения


In [ ]:

print("=" * 60)
print("ПРОПУСКИ В ТРАНЗАКЦИЯХ")
print("=" * 60)
missing_trans = df_trans.isnull().sum()
missing_trans_pct = (missing_trans / len(df_trans) * 100).round(2)
missing_df_trans = pd.DataFrame({
    'Колонка': missing_trans.index,
    'Пропусков': missing_trans.values,
    'Процент': missing_trans_pct.values
})
missing_df_trans = missing_df_trans[missing_df_trans['Пропусков'] > 0].sort_values('Пропусков', ascending=False)
if len(missing_df_trans) > 0:
    print(missing_df_trans.to_string(index=False))
else:
    print(" Пропусков не найдено")

print("\n" + "=" * 60)
print("ПРОПУСКИ В ПОВЕДЕНЧЕСКИХ ДАННЫХ")
print("=" * 60)
missing_behav = df_behav.isnull().sum()
missing_behav_pct = (missing_behav / len(df_behav) * 100).round(2)
missing_df_behav = pd.DataFrame({
    'Колонка': missing_behav.index,
    'Пропусков': missing_behav.values,
    'Процент': missing_behav_pct.values
})
missing_df_behav = missing_df_behav[missing_df_behav['Пропусков'] > 0].sort_values('Пропусков', ascending=False)
if len(missing_df_behav) > 0:
    print(missing_df_behav.to_string(index=False))
else:
    print(" Пропусков не найдено")

# Визуализация пропусков
if len(missing_df_trans) > 0 or len(missing_df_behav) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    if len(missing_df_trans) > 0:
        missing_df_trans.head(10).plot(x='Колонка', y='Процент', kind='barh', ax=axes[0], color='coral')
        axes[0].set_title('Пропуски в транзакциях (топ-10)', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Процент пропусков')
        axes[0].grid(axis='x', alpha=0.3)
    else:
        axes[0].text(0.5, 0.5, 'Нет пропусков', ha='center', va='center', fontsize=14)
        axes[0].set_title('Пропуски в транзакциях', fontsize=12, fontweight='bold')
    
    if len(missing_df_behav) > 0:
        missing_df_behav.head(10).plot(x='Колонка', y='Процент', kind='barh', ax=axes[1], color='skyblue')
        axes[1].set_title('Пропуски в поведенческих данных (топ-10)', fontsize=12, fontweight='bold')
        axes[1].set_xlabel('Процент пропусков')
        axes[1].grid(axis='x', alpha=0.3)
    else:
        axes[1].text(0.5, 0.5, 'Нет пропусков', ha='center', va='center', fontsize=14)
        axes[1].set_title('Пропуски в поведенческих данных', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()


In [ ]:


# Подготовка данных: очистка и преобразование amount
df_trans_clean = df_trans.copy()

# Очистка колонки amount (если нужно)
if 'amount' in df_trans_clean.columns:
    # Преобразование amount в числовой формат
    if df_trans_clean['amount'].dtype == 'object':
        df_trans_clean['amount'] = df_trans_clean['amount'].astype(str).str.replace(r'[\.,]', '', regex=True)
        df_trans_clean['amount'] = pd.to_numeric(df_trans_clean['amount'], errors='coerce')
    
    # Базовая статистика
    print("СТАТИСТИКА ПО AMOUNT:")
    print("=" * 60)
    print(df_trans_clean['amount'].describe())
    print(f"\nМедиана: {df_trans_clean['amount'].median():.2f}")
    print(f"Стандартное отклонение: {df_trans_clean['amount'].std():.2f}")
    print(f"Коэффициент вариации: {(df_trans_clean['amount'].std() / df_trans_clean['amount'].mean() * 100):.2f}%")
    
    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Гистограмма
    axes[0, 0].hist(df_trans_clean['amount'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].set_title('Распределение amount (все данные)', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Amount')
    axes[0, 0].set_ylabel('Частота')
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Логарифмическая шкала (для лучшей визуализации)
    amount_log = np.log1p(df_trans_clean['amount'][df_trans_clean['amount'] > 0])
    axes[0, 1].hist(amount_log, bins=50, edgecolor='black', alpha=0.7, color='coral')
    axes[0, 1].set_title('Распределение log(amount+1)', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('log(Amount + 1)')
    axes[0, 1].set_ylabel('Частота')
    axes[0, 1].grid(alpha=0.3)
    
    # 3. Box plot
    axes[1, 0].boxplot(df_trans_clean['amount'], vert=True, patch_artist=True,
                       boxprops=dict(facecolor='lightblue', alpha=0.7))
    axes[1, 0].set_title('Box Plot: amount', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Amount')
    axes[1, 0].grid(alpha=0.3, axis='y')
    
    # 4. Распределение по классам (если есть target)
    if 'target' in df_trans_clean.columns:
        fraud_amount = df_trans_clean[df_trans_clean['target'] == 1]['amount']
        normal_amount = df_trans_clean[df_trans_clean['target'] == 0]['amount']
        
        axes[1, 1].hist([normal_amount, fraud_amount], bins=30, alpha=0.7, 
                        label=['Нормальные', 'Мошеннические'], color=['green', 'red'], edgecolor='black')
        axes[1, 1].set_title('Распределение amount по классам', fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Amount')
        axes[1, 1].set_ylabel('Частота')
        axes[1, 1].legend()
        axes[1, 1].grid(alpha=0.3)
        
        print(f"\nСРАВНЕНИЕ ПО КЛАССАМ:")
        print("=" * 60)
        print(f"Нормальные транзакции:")
        print(f"  Среднее: {normal_amount.mean():.2f}")
        print(f"  Медиана: {normal_amount.median():.2f}")
        print(f"\nМошеннические транзакции:")
        print(f"  Среднее: {fraud_amount.mean():.2f}")
        print(f"  Медиана: {fraud_amount.median():.2f}")
    else:
        axes[1, 1].text(0.5, 0.5, 'Колонка target не найдена', ha='center', va='center', fontsize=12)
        axes[1, 1].set_title('Распределение по классам', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print(" Колонка 'amount' не найдена в данных")


In [ ]:

# Подготовка временных меток
df_time = df_trans_clean.copy()

# Поиск колонок с датами/временем
date_cols = []
for col in df_time.columns:
    if 'date' in col.lower() or 'time' in col.lower() or 'datetime' in col.lower():
        date_cols.append(col)

print(f"Найденные временные колонки: {date_cols}")

# Преобразование в datetime
if len(date_cols) > 0:
    for col in date_cols:
        try:
            df_time[col] = pd.to_datetime(df_time[col], errors='coerce')
            print(f" {col} преобразована в datetime")
        except:
            print(f" Не удалось преобразовать {col}")
    
    # Используем первую доступную временную колонку
    time_col = date_cols[0]
    df_time = df_time[df_time[time_col].notna()].copy()
    
    # Извлечение временных признаков
    df_time['hour'] = df_time[time_col].dt.hour
    df_time['day_of_week'] = df_time[time_col].dt.dayofweek
    df_time['day_name'] = df_time[time_col].dt.day_name()
    df_time['month'] = df_time[time_col].dt.month
    df_time['date'] = df_time[time_col].dt.date
    
    print(f"\nВременной диапазон:")
    print(f"  Начало: {df_time[time_col].min()}")
    print(f"  Конец: {df_time[time_col].max()}")
    print(f"  Длительность: {df_time[time_col].max() - df_time[time_col].min()}")
    
    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Распределение по часам
    hour_counts = df_time['hour'].value_counts().sort_index()
    axes[0, 0].bar(hour_counts.index, hour_counts.values, color='steelblue', alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Распределение транзакций по часам дня', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Час дня')
    axes[0, 0].set_ylabel('Количество транзакций')
    axes[0, 0].grid(alpha=0.3, axis='y')
    axes[0, 0].set_xticks(range(0, 24, 2))
    
    # 2. Распределение по дням недели
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = df_time['day_name'].value_counts().reindex(day_order)
    axes[0, 1].bar(range(len(day_counts)), day_counts.values, color='coral', alpha=0.7, edgecolor='black')
    axes[0, 1].set_title('Распределение транзакций по дням недели', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('День недели')
    axes[0, 1].set_ylabel('Количество транзакций')
    axes[0, 1].set_xticks(range(len(day_counts)))
    axes[0, 1].set_xticklabels(day_counts.index, rotation=45, ha='right')
    axes[0, 1].grid(alpha=0.3, axis='y')
    
    # 3. Временной ряд (транзакции по датам)
    daily_counts = df_time.groupby('date').size()
    axes[1, 0].plot(daily_counts.index, daily_counts.values, marker='o', linewidth=2, markersize=4, color='green')
    axes[1, 0].set_title('Количество транзакций по дням', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Дата')
    axes[1, 0].set_ylabel('Количество транзакций')
    axes[1, 0].grid(alpha=0.3)
    axes[1, 0].tick_params(axis='x', rotation=45)
    
    # 4. Распределение по часам с разделением по классам (если есть target)
    if 'target' in df_time.columns:
        fraud_hours = df_time[df_time['target'] == 1]['hour'].value_counts().sort_index()
        normal_hours = df_time[df_time['target'] == 0]['hour'].value_counts().sort_index()
        
        x = range(24)
        width = 0.35
        axes[1, 1].bar([i - width/2 for i in x], [normal_hours.get(i, 0) for i in x], 
                       width, label='Нормальные', color='green', alpha=0.7, edgecolor='black')
        axes[1, 1].bar([i + width/2 for i in x], [fraud_hours.get(i, 0) for i in x], 
                       width, label='Мошеннические', color='red', alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Распределение по часам (по классам)', fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Час дня')
        axes[1, 1].set_ylabel('Количество транзакций')
        axes[1, 1].set_xticks(range(0, 24, 2))
        axes[1, 1].legend()
        axes[1, 1].grid(alpha=0.3, axis='y')
    else:
        axes[1, 1].text(0.5, 0.5, 'Колонка target не найдена', ha='center', va='center', fontsize=12)
        axes[1, 1].set_title('Распределение по часам (по классам)', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nСТАТИСТИКА ПО ВРЕМЕНИ:")
    print("=" * 60)
    print(f"Самый активный час: {hour_counts.idxmax()} ({hour_counts.max()} транзакций)")
    print(f"Самый активный день недели: {day_counts.idxmax()} ({day_counts.max()} транзакций)")
else:
    print(" Временные колонки не найдены в данных")


In [ ]:

if 'target' in df_trans_clean.columns:
    # Базовая статистика
    class_counts = df_trans_clean['target'].value_counts().sort_index()
    class_pct = df_trans_clean['target'].value_counts(normalize=True).sort_index() * 100
    
    print("РАСПРЕДЕЛЕНИЕ КЛАССОВ:")
    print("=" * 60)
    for cls in class_counts.index:
        cls_name = 'Мошеннические' if cls == 1 else 'Нормальные'
        print(f"{cls_name} (класс {cls}):")
        print(f"  Количество: {class_counts[cls]:,}")
        print(f"  Процент: {class_pct[cls]:.2f}%")
    
    # Коэффициент дисбаланса
    imbalance_ratio = class_counts[0] / class_counts[1] if class_counts[1] > 0 else 0
    print(f"\nКоэффициент дисбаланса (нормальные/мошеннические): {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 10:
        print(" ВНИМАНИЕ: Сильный дисбаланс классов! Рекомендуется использовать:")
        print("   - Метрики: Precision, Recall, F1-score, ROC-AUC")
        print("   - Техники: SMOTE, undersampling, class_weight в моделях")
    
    # Визуализация
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 1. Столбчатая диаграмма
    classes = ['Нормальные\n(0)', 'Мошеннические\n(1)']
    colors = ['green', 'red']
    bars = axes[0].bar(classes, class_counts.values, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    axes[0].set_title('Распределение классов', fontsize=12, fontweight='bold')
    axes[0].set_ylabel('Количество транзакций')
    axes[0].grid(alpha=0.3, axis='y')
    
    # Добавление значений на столбцы
    for bar, count, pct in zip(bars, class_counts.values, class_pct.values):
        height = bar.get_height()
        axes[0].text(bar.get_x() + bar.get_width()/2., height,
                    f'{count:,}\n({pct:.1f}%)',
                    ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # 2. Круговая диаграмма
    axes[1].pie(class_counts.values, labels=classes, autopct='%1.1f%%', 
               colors=colors, startangle=90, explode=(0.05, 0.1), shadow=True)
    axes[1].set_title('Процентное распределение классов', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Дополнительный анализ: распределение amount по классам
    if 'amount' in df_trans_clean.columns:
        print(f"\nСРАВНИТЕЛЬНАЯ СТАТИСТИКА AMOUNT ПО КЛАССАМ:")
        print("=" * 60)
        comparison = df_trans_clean.groupby('target')['amount'].agg(['count', 'mean', 'median', 'std', 'min', 'max'])
        comparison.index = ['Нормальные', 'Мошеннические']
        print(comparison.round(2))
        
        # Визуализация сравнения
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Box plot по классам
        fraud_amount = df_trans_clean[df_trans_clean['target'] == 1]['amount']
        normal_amount = df_trans_clean[df_trans_clean['target'] == 0]['amount']
        
        axes[0].boxplot([normal_amount, fraud_amount], labels=['Нормальные', 'Мошеннические'],
                       patch_artist=True, 
                       boxprops=[dict(facecolor='green', alpha=0.7), dict(facecolor='red', alpha=0.7)])
        axes[0].set_title('Распределение amount по классам (Box Plot)', fontsize=12, fontweight='bold')
        axes[0].set_ylabel('Amount')
        axes[0].grid(alpha=0.3, axis='y')
        
        # Сравнение средних значений
        means = [normal_amount.mean(), fraud_amount.mean()]
        stds = [normal_amount.std(), fraud_amount.std()]
        x_pos = [0, 1]
        axes[1].bar(x_pos, means, yerr=stds, color=['green', 'red'], alpha=0.7, 
                   edgecolor='black', linewidth=2, capsize=10)
        axes[1].set_xticks(x_pos)
        axes[1].set_xticklabels(['Нормальные', 'Мошеннические'])
        axes[1].set_title('Среднее значение amount по классам', fontsize=12, fontweight='bold')
        axes[1].set_ylabel('Amount (среднее ± std)')
        axes[1].grid(alpha=0.3, axis='y')
        
        # Добавление значений
        for i, (mean, std) in enumerate(zip(means, stds)):
            axes[1].text(i, mean + std + max(means) * 0.05, f'{mean:.2f}', 
                       ha='center', fontweight='bold', fontsize=10)
        
        plt.tight_layout()
        plt.show()
else:
    print(" Колонка 'target' не найдена в данных")
    print("   Это нормально для EDA на этапе загрузки данных")


In [ ]:

# Подготовка данных: нужно объединить транзакции и поведенческие данные для анализа по пользователям
# Сначала подготовим данные для анализа

# Очистка и нормализация имен колонок (если еще не сделано)
def clean_cols(df):
    df.columns = df.columns.astype(str).str.replace(' ', '_').str.lower().str.replace('"', '').str.strip()
    df.columns = df.columns.str.replace(r'[^\w]+', '_', regex=True).str.strip('_')
    return df

df_trans_clean_cols = clean_cols(df_trans_clean.copy())
df_behav_clean = clean_cols(df_behav.copy())

# Поиск колонки с ID пользователя
user_id_col_trans = None
for col in df_trans_clean_cols.columns:
    if 'user' in col.lower() or 'id' in col.lower() or 'cst' in col.lower() or 'клиент' in col.lower():
        if user_id_col_trans is None:
            user_id_col_trans = col
            break

user_id_col_behav = None
for col in df_behav_clean.columns:
    if 'user' in col.lower() or 'id' in col.lower() or 'клиент' in col.lower() or 'уникальный' in col.lower():
        if user_id_col_behav is None:
            user_id_col_behav = col
            break

print(f"Колонка ID пользователя в транзакциях: {user_id_col_trans}")
print(f"Колонка ID пользователя в поведенческих данных: {user_id_col_behav}")

if user_id_col_trans:
    # Анализ по пользователям в транзакциях
    print("\n" + "=" * 60)
    print("АНАЛИЗ ПО ПОЛЬЗОВАТЕЛЯМ (ТРАНЗАКЦИИ)")
    print("=" * 60)
    
    # Статистика по пользователям
    user_stats = df_trans_clean_cols.groupby(user_id_col_trans).agg({
        'amount': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    }).round(2)
    user_stats.columns = ['tx_count', 'total_amount', 'avg_amount', 'std_amount', 'min_amount', 'max_amount']
    
    print(f"\nОбщая статистика:")
    print(f"  Всего уникальных пользователей: {df_trans_clean_cols[user_id_col_trans].nunique():,}")
    print(f"  Всего транзакций: {len(df_trans_clean_cols):,}")
    print(f"  Среднее транзакций на пользователя: {user_stats['tx_count'].mean():.2f}")
    print(f"  Медиана транзакций на пользователя: {user_stats['tx_count'].median():.2f}")
    
    # Топ пользователей
    print(f"\nТОП-10 пользователей по количеству транзакций:")
    top_users_tx = user_stats.nlargest(10, 'tx_count')[['tx_count', 'total_amount', 'avg_amount']]
    print(top_users_tx)
    
    print(f"\nТОП-10 пользователей по общей сумме транзакций:")
    top_users_amount = user_stats.nlargest(10, 'total_amount')[['tx_count', 'total_amount', 'avg_amount']]
    print(top_users_amount)
    
    # Анализ мошенничества по пользователям (если есть target)
    if 'target' in df_trans_clean_cols.columns:
        fraud_by_user = df_trans_clean_cols.groupby(user_id_col_trans).agg({
            'target': ['sum', 'count']
        })
        fraud_by_user.columns = ['fraud_count', 'total_tx']
        fraud_by_user['fraud_rate'] = (fraud_by_user['fraud_count'] / fraud_by_user['total_tx'] * 100).round(2)
        fraud_by_user = fraud_by_user[fraud_by_user['fraud_count'] > 0].sort_values('fraud_count', ascending=False)
        
        print(f"\nПОЛЬЗОВАТЕЛИ С МОШЕННИЧЕСКИМИ ТРАНЗАКЦИЯМИ:")
        print(f"  Всего пользователей с мошенничеством: {len(fraud_by_user):,}")
        print(f"\nТОП-10 пользователей по количеству мошеннических транзакций:")
        print(fraud_by_user.head(10))
    
    # Визуализация
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. Распределение количества транзакций на пользователя
    axes[0, 0].hist(user_stats['tx_count'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    axes[0, 0].set_title('Распределение количества транзакций на пользователя', fontsize=12, fontweight='bold')
    axes[0, 0].set_xlabel('Количество транзакций')
    axes[0, 0].set_ylabel('Количество пользователей')
    axes[0, 0].grid(alpha=0.3)
    
    # 2. Распределение средней суммы транзакций на пользователя
    axes[0, 1].hist(user_stats['avg_amount'], bins=50, edgecolor='black', alpha=0.7, color='coral')
    axes[0, 1].set_title('Распределение средней суммы транзакций на пользователя', fontsize=12, fontweight='bold')
    axes[0, 1].set_xlabel('Средняя сумма транзакции')
    axes[0, 1].set_ylabel('Количество пользователей')
    axes[0, 1].grid(alpha=0.3)
    
    # 3. Топ-15 пользователей по количеству транзакций
    top_15 = user_stats.nlargest(15, 'tx_count')
    axes[1, 0].barh(range(len(top_15)), top_15['tx_count'].values, color='green', alpha=0.7, edgecolor='black')
    axes[1, 0].set_yticks(range(len(top_15)))
    axes[1, 0].set_yticklabels([str(idx) for idx in top_15.index], fontsize=8)
    axes[1, 0].set_title('ТОП-15 пользователей по количеству транзакций', fontsize=12, fontweight='bold')
    axes[1, 0].set_xlabel('Количество транзакций')
    axes[1, 0].grid(alpha=0.3, axis='x')
    axes[1, 0].invert_yaxis()
    
    # 4. Распределение мошенничества по пользователям (если есть target)
    if 'target' in df_trans_clean_cols.columns:
        fraud_users_dist = fraud_by_user['fraud_count'].value_counts().sort_index()
        axes[1, 1].bar(fraud_users_dist.index, fraud_users_dist.values, color='red', alpha=0.7, edgecolor='black')
        axes[1, 1].set_title('Распределение количества мошеннических транзакций на пользователя', fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Количество мошеннических транзакций')
        axes[1, 1].set_ylabel('Количество пользователей')
        axes[1, 1].grid(alpha=0.3, axis='y')
    else:
        axes[1, 1].text(0.5, 0.5, 'Колонка target не найдена', ha='center', va='center', fontsize=12)
        axes[1, 1].set_title('Распределение мошенничества', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print(" Колонка с ID пользователя не найдена в транзакциях")


In [ ]:

# Поиск колонок, связанных с устройствами и IP
device_cols = []
ip_cols = []

# Поиск в транзакциях
for col in df_trans_clean_cols.columns:
    col_lower = col.lower()
    if 'device' in col_lower or 'phone' in col_lower or 'model' in col_lower or 'os' in col_lower or 'устройство' in col_lower or 'телефон' in col_lower:
        device_cols.append(col)
    if 'ip' in col_lower or 'адрес' in col_lower:
        ip_cols.append(col)

# Поиск в поведенческих данных
for col in df_behav_clean.columns:
    col_lower = col.lower()
    if 'device' in col_lower or 'phone' in col_lower or 'model' in col_lower or 'os' in col_lower or 'устройство' in col_lower or 'телефон' in col_lower:
        if col not in device_cols:
            device_cols.append(col)
    if 'ip' in col_lower or 'адрес' in col_lower:
        if col not in ip_cols:
            ip_cols.append(col)

print(f"Найденные колонки с устройствами: {device_cols}")
print(f"Найденные колонки с IP: {ip_cols}")

# Анализ устройств
if len(device_cols) > 0:
    print("\n" + "=" * 60)
    print("АНАЛИЗ ПО УСТРОЙСТВАМ")
    print("=" * 60)
    
    # Используем первую найденную колонку с устройствами
    device_col = device_cols[0]
    
    # Объединяем данные для анализа
    df_merged_analysis = df_trans_clean_cols.copy()
    
    # Если колонка есть в поведенческих данных, объединяем
    if device_col in df_behav_clean.columns and user_id_col_trans and user_id_col_behav:
        # Простое объединение по первой доступной колонке устройства
        pass  # Пока используем только транзакции
    
    if device_col in df_merged_analysis.columns:
        device_stats = df_merged_analysis[device_col].value_counts()
        
        print(f"\nСтатистика по устройствам:")
        print(f"  Всего уникальных устройств: {df_merged_analysis[device_col].nunique():,}")
        print(f"  Всего записей: {len(df_merged_analysis):,}")
        print(f"\nТОП-10 устройств по частоте использования:")
        print(device_stats.head(10))
        
        # Анализ мошенничества по устройствам
        if 'target' in df_merged_analysis.columns:
            fraud_by_device = df_merged_analysis.groupby(device_col).agg({
                'target': ['sum', 'count']
            })
            fraud_by_device.columns = ['fraud_count', 'total_tx']
            fraud_by_device['fraud_rate'] = (fraud_by_device['fraud_count'] / fraud_by_device['total_tx'] * 100).round(2)
            fraud_by_device = fraud_by_device[fraud_by_device['fraud_count'] > 0].sort_values('fraud_rate', ascending=False)
            
            print(f"\nУСТРОЙСТВА С МОШЕННИЧЕСКИМИ ТРАНЗАКЦИЯМИ:")
            print(f"  Всего устройств с мошенничеством: {len(fraud_by_device):,}")
            print(f"\nТОП-10 устройств по проценту мошеннических транзакций:")
            print(fraud_by_device.head(10))
        
        # Визуализация
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # 1. Топ-15 устройств по частоте использования
        top_devices = device_stats.head(15)
        axes[0, 0].barh(range(len(top_devices)), top_devices.values, color='steelblue', alpha=0.7, edgecolor='black')
        axes[0, 0].set_yticks(range(len(top_devices)))
        axes[0, 0].set_yticklabels([str(dev)[:30] for dev in top_devices.index], fontsize=8)
        axes[0, 0].set_title('ТОП-15 устройств по частоте использования', fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Количество транзакций')
        axes[0, 0].grid(alpha=0.3, axis='x')
        axes[0, 0].invert_yaxis()
        
        # 2. Распределение количества транзакций на устройство
        device_tx_counts = df_merged_analysis.groupby(device_col).size()
        axes[0, 1].hist(device_tx_counts, bins=50, edgecolor='black', alpha=0.7, color='coral')
        axes[0, 1].set_title('Распределение количества транзакций на устройство', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Количество транзакций')
        axes[0, 1].set_ylabel('Количество устройств')
        axes[0, 1].grid(alpha=0.3)
        
        # 3. Процент мошенничества по устройствам (если есть target)
        if 'target' in df_merged_analysis.columns and len(fraud_by_device) > 0:
            top_fraud_devices = fraud_by_device.head(15)
            axes[1, 0].barh(range(len(top_fraud_devices)), top_fraud_devices['fraud_rate'].values, 
                           color='red', alpha=0.7, edgecolor='black')
            axes[1, 0].set_yticks(range(len(top_fraud_devices)))
            axes[1, 0].set_yticklabels([str(dev)[:30] for dev in top_fraud_devices.index], fontsize=8)
            axes[1, 0].set_title('ТОП-15 устройств по проценту мошенничества', fontsize=12, fontweight='bold')
            axes[1, 0].set_xlabel('Процент мошеннических транзакций (%)')
            axes[1, 0].grid(alpha=0.3, axis='x')
            axes[1, 0].invert_yaxis()
        else:
            axes[1, 0].text(0.5, 0.5, 'Нет данных о мошенничестве', ha='center', va='center', fontsize=12)
            axes[1, 0].set_title('Мошенничество по устройствам', fontsize=12, fontweight='bold')
        
        # 4. Круговая диаграмма топ-10 устройств
        top_10_devices = device_stats.head(10)
        other_count = device_stats.iloc[10:].sum() if len(device_stats) > 10 else 0
        if other_count > 0:
            top_10_devices['Другие'] = other_count
        
        axes[1, 1].pie(top_10_devices.values, labels=[str(dev)[:20] for dev in top_10_devices.index], 
                      autopct='%1.1f%%', startangle=90)
        axes[1, 1].set_title('Распределение транзакций по устройствам (ТОП-10)', fontsize=12, fontweight='bold')
        
        plt.tight_layout()
        plt.show()
    else:
        print(f" Колонка {device_col} не найдена в объединенных данных")
else:
    print(" Колонки с устройствами не найдены в данных")

# Анализ IP адресов
if len(ip_cols) > 0:
    print("\n" + "=" * 60)
    print("АНАЛИЗ ПО IP АДРЕСАМ")
    print("=" * 60)
    
    ip_col = ip_cols[0]
    if ip_col in df_merged_analysis.columns:
        ip_stats = df_merged_analysis[ip_col].value_counts()
        
        print(f"\nСтатистика по IP адресам:")
        print(f"  Всего уникальных IP: {df_merged_analysis[ip_col].nunique():,}")
        print(f"\nТОП-10 IP адресов по частоте использования:")
        print(ip_stats.head(10))
        
        # Анализ мошенничества по IP
        if 'target' in df_merged_analysis.columns:
            fraud_by_ip = df_merged_analysis.groupby(ip_col).agg({
                'target': ['sum', 'count']
            })
            fraud_by_ip.columns = ['fraud_count', 'total_tx']
            fraud_by_ip['fraud_rate'] = (fraud_by_ip['fraud_count'] / fraud_by_ip['total_tx'] * 100).round(2)
            fraud_by_ip = fraud_by_ip[fraud_by_ip['fraud_count'] > 0].sort_values('fraud_rate', ascending=False)
            
            print(f"\nIP АДРЕСА С МОШЕННИЧЕСКИМИ ТРАНЗАКЦИЯМИ:")
            print(f"  Всего IP с мошенничеством: {len(fraud_by_ip):,}")
            print(f"\nТОП-10 IP по проценту мошеннических транзакций:")
            print(fraud_by_ip.head(10))
    else:
        print(f" Колонка {ip_col} не найдена в данных")
else:
    print("\n Колонки с IP адресами не найдены в данных")
    print("   Это нормально, если IP адреса не включены в датасет")


In [ ]:

# Создаем комплексную визуализацию основных метрик
fig = plt.figure(figsize=(18, 12))

# Подготовка данных
df_viz = df_trans_clean_cols.copy()

# 1. Распределение amount (верхний левый)
ax1 = plt.subplot(3, 3, 1)
if 'amount' in df_viz.columns:
    ax1.hist(df_viz['amount'], bins=50, edgecolor='black', alpha=0.7, color='steelblue')
    ax1.set_title('Распределение Amount', fontsize=11, fontweight='bold')
    ax1.set_xlabel('Amount')
    ax1.set_ylabel('Частота')
    ax1.grid(alpha=0.3)

# 2. Распределение по часам (верхний средний)
ax2 = plt.subplot(3, 3, 2)
if 'timestamp' in df_viz.columns or any('time' in col.lower() or 'date' in col.lower() for col in df_viz.columns):
    time_col = 'timestamp' if 'timestamp' in df_viz.columns else [col for col in df_viz.columns if 'time' in col.lower() or 'date' in col.lower()][0]
    df_viz[time_col] = pd.to_datetime(df_viz[time_col], errors='coerce')
    hour_counts = df_viz[df_viz[time_col].notna()][time_col].dt.hour.value_counts().sort_index()
    ax2.bar(hour_counts.index, hour_counts.values, color='coral', alpha=0.7, edgecolor='black')
    ax2.set_title('Распределение по часам', fontsize=11, fontweight='bold')
    ax2.set_xlabel('Час дня')
    ax2.set_ylabel('Количество транзакций')
    ax2.grid(alpha=0.3, axis='y')

# 3. Баланс классов (верхний правый)
ax3 = plt.subplot(3, 3, 3)
if 'target' in df_viz.columns:
    class_counts = df_viz['target'].value_counts().sort_index()
    colors = ['green', 'red']
    labels = ['Нормальные', 'Мошеннические']
    ax3.pie(class_counts.values, labels=labels, autopct='%1.1f%%', colors=colors, startangle=90)
    ax3.set_title('Баланс классов', fontsize=11, fontweight='bold')
else:
    ax3.text(0.5, 0.5, 'Нет данных о классах', ha='center', va='center', fontsize=11)
    ax3.set_title('Баланс классов', fontsize=11, fontweight='bold')

# 4. Amount по классам (Box Plot) - средний левый
ax4 = plt.subplot(3, 3, 4)
if 'target' in df_viz.columns and 'amount' in df_viz.columns:
    fraud_amount = df_viz[df_viz['target'] == 1]['amount']
    normal_amount = df_viz[df_viz['target'] == 0]['amount']
    ax4.boxplot([normal_amount, fraud_amount], labels=['Нормальные', 'Мошеннические'],
               patch_artist=True,
               boxprops=[dict(facecolor='green', alpha=0.7), dict(facecolor='red', alpha=0.7)])
    ax4.set_title('Amount по классам (Box Plot)', fontsize=11, fontweight='bold')
    ax4.set_ylabel('Amount')
    ax4.grid(alpha=0.3, axis='y')

# 5. Временной ряд транзакций (средний средний)
ax5 = plt.subplot(3, 3, 5)
if 'timestamp' in df_viz.columns or any('time' in col.lower() or 'date' in col.lower() for col in df_viz.columns):
    time_col = 'timestamp' if 'timestamp' in df_viz.columns else [col for col in df_viz.columns if 'time' in col.lower() or 'date' in col.lower()][0]
    df_viz[time_col] = pd.to_datetime(df_viz[time_col], errors='coerce')
    daily_counts = df_viz[df_viz[time_col].notna()].groupby(df_viz[df_viz[time_col].notna()][time_col].dt.date).size()
    ax5.plot(range(len(daily_counts)), daily_counts.values, marker='o', linewidth=2, markersize=3, color='green')
    ax5.set_title('Временной ряд транзакций', fontsize=11, fontweight='bold')
    ax5.set_xlabel('День')
    ax5.set_ylabel('Количество транзакций')
    ax5.grid(alpha=0.3)

# 6. Распределение по дням недели (средний правый)
ax6 = plt.subplot(3, 3, 6)
if 'timestamp' in df_viz.columns or any('time' in col.lower() or 'date' in col.lower() for col in df_viz.columns):
    time_col = 'timestamp' if 'timestamp' in df_viz.columns else [col for col in df_viz.columns if 'time' in col.lower() or 'date' in col.lower()][0]
    df_viz[time_col] = pd.to_datetime(df_viz[time_col], errors='coerce')
    day_counts = df_viz[df_viz[time_col].notna()][time_col].dt.day_name().value_counts()
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
    day_counts = day_counts.reindex([d for d in day_order if d in day_counts.index])
    ax6.bar(range(len(day_counts)), day_counts.values, color='purple', alpha=0.7, edgecolor='black')
    ax6.set_xticks(range(len(day_counts)))
    ax6.set_xticklabels(day_counts.index, rotation=45, ha='right', fontsize=9)
    ax6.set_title('Распределение по дням недели', fontsize=11, fontweight='bold')
    ax6.set_ylabel('Количество транзакций')
    ax6.grid(alpha=0.3, axis='y')

# 7. Статистика по пользователям (нижний левый)
ax7 = plt.subplot(3, 3, 7)
if user_id_col_trans:
    user_tx_counts = df_viz.groupby(user_id_col_trans).size()
    ax7.hist(user_tx_counts, bins=50, edgecolor='black', alpha=0.7, color='orange')
    ax7.set_title('Распределение транзакций на пользователя', fontsize=11, fontweight='bold')
    ax7.set_xlabel('Количество транзакций')
    ax7.set_ylabel('Количество пользователей')
    ax7.grid(alpha=0.3)

# 8. Мошенничество по часам (нижний средний)
ax8 = plt.subplot(3, 3, 8)
if 'target' in df_viz.columns and ('timestamp' in df_viz.columns or any('time' in col.lower() or 'date' in col.lower() for col in df_viz.columns)):
    time_col = 'timestamp' if 'timestamp' in df_viz.columns else [col for col in df_viz.columns if 'time' in col.lower() or 'date' in col.lower()][0]
    df_viz[time_col] = pd.to_datetime(df_viz[time_col], errors='coerce')
    fraud_hours = df_viz[(df_viz['target'] == 1) & (df_viz[time_col].notna())][time_col].dt.hour.value_counts().sort_index()
    normal_hours = df_viz[(df_viz['target'] == 0) & (df_viz[time_col].notna())][time_col].dt.hour.value_counts().sort_index()
    
    x = range(24)
    width = 0.35
    ax8.bar([i - width/2 for i in x], [normal_hours.get(i, 0) for i in x], 
           width, label='Нормальные', color='green', alpha=0.7)
    ax8.bar([i + width/2 for i in x], [fraud_hours.get(i, 0) for i in x], 
           width, label='Мошеннические', color='red', alpha=0.7)
    ax8.set_title('Распределение по часам (по классам)', fontsize=11, fontweight='bold')
    ax8.set_xlabel('Час дня')
    ax8.set_ylabel('Количество транзакций')
    ax8.set_xticks(range(0, 24, 2))
    ax8.legend(fontsize=9)
    ax8.grid(alpha=0.3, axis='y')

# 9. Сводная статистика (нижний правый)
ax9 = plt.subplot(3, 3, 9)
ax9.axis('off')
stats_text = "СВОДНАЯ СТАТИСТИКА\n" + "=" * 30 + "\n\n"
stats_text += f"Всего транзакций: {len(df_viz):,}\n"
if user_id_col_trans:
    stats_text += f"Уникальных пользователей: {df_viz[user_id_col_trans].nunique():,}\n"
if 'amount' in df_viz.columns:
    stats_text += f"Средний amount: {df_viz['amount'].mean():.2f}\n"
    stats_text += f"Медиана amount: {df_viz['amount'].median():.2f}\n"
if 'target' in df_viz.columns:
    fraud_count = (df_viz['target'] == 1).sum()
    fraud_pct = (fraud_count / len(df_viz) * 100)
    stats_text += f"\nМошеннических: {fraud_count:,} ({fraud_pct:.2f}%)\n"
    normal_count = (df_viz['target'] == 0).sum()
    stats_text += f"Нормальных: {normal_count:,} ({100-fraud_pct:.2f}%)\n"
ax9.text(0.1, 0.5, stats_text, fontsize=10, verticalalignment='center', 
         family='monospace', fontweight='bold')

plt.suptitle('ВИЗУАЛИЗАЦИЯ ОСНОВНЫХ МЕТРИК ДАТАСЕТА', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout(rect=[0, 0, 1, 0.99])
plt.show()

print(" Комплексная визуализация основных метрик создана")


In [ ]:

# Подготовка данных для корреляционного анализа
df_corr = df_trans_clean_cols.copy()

# Выбираем только числовые колонки
numeric_cols = df_corr.select_dtypes(include=[np.number]).columns.tolist()

# Исключаем колонки, которые не имеют смысла для корреляции (например, ID)
exclude_cols = []
for col in numeric_cols:
    if 'id' in col.lower() or col.lower() == 'target':  # target оставляем для анализа корреляции с ним
        if 'id' in col.lower():
            exclude_cols.append(col)

numeric_cols = [col for col in numeric_cols if col not in exclude_cols]

# Добавляем target в конец для удобства анализа
if 'target' in df_corr.columns:
    numeric_cols.append('target')

print(f"Колонки для корреляционного анализа ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols[:20], 1):  # Показываем первые 20
    print(f"  {i}. {col}")
if len(numeric_cols) > 20:
    print(f"  ... и еще {len(numeric_cols) - 20} колонок")

# Создаем корреляционную матрицу
df_corr_numeric = df_corr[numeric_cols].copy()

# Заполняем пропуски перед расчетом корреляции
df_corr_numeric = df_corr_numeric.fillna(0)

# Вычисляем корреляцию
correlation_matrix = df_corr_numeric.corr()

print(f"\nРазмер корреляционной матрицы: {correlation_matrix.shape}")

# Анализ корреляции с target (если есть)
if 'target' in correlation_matrix.columns:
    print("\n" + "=" * 60)
    print("ТОП-15 ПРИЗНАКОВ С НАИБОЛЬШЕЙ КОРРЕЛЯЦИЕЙ С TARGET")
    print("=" * 60)
    
    target_corr = correlation_matrix['target'].drop('target').abs().sort_values(ascending=False)
    top_corr = target_corr.head(15)
    
    print("\nПризнаки с наибольшей корреляцией (по абсолютному значению):")
    for i, (feature, corr_value) in enumerate(top_corr.items(), 1):
        actual_corr = correlation_matrix.loc[feature, 'target']
        direction = "положительная" if actual_corr > 0 else "отрицательная"
        print(f"  {i:2d}. {feature[:50]:50s} | {actual_corr:7.4f} ({direction})")

# Визуализация корреляционной матрицы
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# 1. Полная корреляционная матрица (ограничиваем до разумного количества колонок для визуализации)
max_cols_for_viz = 30
if len(correlation_matrix) > max_cols_for_viz:
    # Берем топ признаков по корреляции с target + сам target
    if 'target' in correlation_matrix.columns:
        target_corr_sorted = correlation_matrix['target'].drop('target').abs().sort_values(ascending=False)
        top_features = target_corr_sorted.head(max_cols_for_viz - 1).index.tolist()
        top_features.append('target')
        corr_subset = correlation_matrix.loc[top_features, top_features]
    else:
        # Если нет target, берем первые max_cols_for_viz колонок
        corr_subset = correlation_matrix.iloc[:max_cols_for_viz, :max_cols_for_viz]
else:
    corr_subset = correlation_matrix

# Heatmap полной матрицы
sns.heatmap(corr_subset, annot=False, cmap='coolwarm', center=0, 
           square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=axes[0])
axes[0].set_title(f'Корреляционная матрица (топ-{len(corr_subset)} признаков)', 
                 fontsize=12, fontweight='bold')
axes[0].tick_params(axis='x', rotation=90, labelsize=8)
axes[0].tick_params(axis='y', rotation=0, labelsize=8)

# 2. Корреляция с target (если есть)
if 'target' in correlation_matrix.columns:
    target_corr_sorted = correlation_matrix['target'].drop('target').sort_values(ascending=False)
    top_20_corr = target_corr_sorted.head(20)
    
    colors = ['red' if x < 0 else 'green' for x in top_20_corr.values]
    axes[1].barh(range(len(top_20_corr)), top_20_corr.values, color=colors, alpha=0.7, edgecolor='black')
    axes[1].set_yticks(range(len(top_20_corr)))
    axes[1].set_yticklabels([str(feat)[:40] for feat in top_20_corr.index], fontsize=9)
    axes[1].set_title('ТОП-20 признаков по корреляции с target', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Корреляция с target')
    axes[1].axvline(x=0, color='black', linestyle='--', linewidth=1)
    axes[1].grid(alpha=0.3, axis='x')
    axes[1].invert_yaxis()
    
    # Добавление значений на столбцы
    for i, (feat, corr_val) in enumerate(top_20_corr.items()):
        axes[1].text(corr_val + (0.01 if corr_val > 0 else -0.01), i, 
                   f'{corr_val:.3f}', va='center', 
                   ha='left' if corr_val > 0 else 'right', fontsize=8, fontweight='bold')
else:
    axes[1].text(0.5, 0.5, 'Колонка target не найдена\nдля анализа корреляции', 
                ha='center', va='center', fontsize=12)
    axes[1].set_title('Корреляция с target', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()



# Находим пары признаков с высокой корреляцией (выше 0.8 или ниже -0.8)
high_corr_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        col1 = correlation_matrix.columns[i]
        col2 = correlation_matrix.columns[j]
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.8 and col1 != 'target' and col2 != 'target':
            high_corr_pairs.append((col1, col2, corr_val))

if len(high_corr_pairs) > 0:
    print(f"\nНайдено {len(high_corr_pairs)} пар признаков с высокой корреляцией (|r| > 0.8):")
    high_corr_pairs_sorted = sorted(high_corr_pairs, key=lambda x: abs(x[2]), reverse=True)
    for i, (col1, col2, corr_val) in enumerate(high_corr_pairs_sorted[:10], 1):
        print(f"  {i:2d}. {col1[:30]:30s} <-> {col2[:30]:30s} | {corr_val:7.4f}")
    if len(high_corr_pairs_sorted) > 10:
        print(f"  ... и еще {len(high_corr_pairs_sorted) - 10} пар")
    print("\n ВНИМАНИЕ: Высокая корреляция между признаками может указывать на мультиколлинеарность.")
    print("   Рекомендуется удалить один из признаков или использовать регуляризацию.")
else:
    print("\n Признаков с высокой корреляцией (|r| > 0.8) не найдено.")

print("\n Корреляционный анализ завершен")
